# Segmenting and Clustering in Toronto
This notebook is for submission to the IBM Data Science Professional Certificate capstone, Week 3 assignment. Data will be scraped from the Toronto Wikipedia page.

## Section 1: Scraping the data, creating the dataframe, and cleaning the data.
### Scraping the data from the website.

In [74]:
import pandas as pd

# Site URL
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

# Write to dataframe
df = pd.read_html(url, header=0)

In [75]:
len(df)

3

This implies that pandas read three tables from this url. Upon inspection, we see that we want the first table (index=0).

In [76]:
df[0].head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


We define our dataframe to be the first table.

In [77]:
df = df[0]
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [78]:
df.shape

(180, 3)

### Cleaning and formatting the data.

In [79]:
df.groupby('Borough').count()

,Postal Code,Neighborhood
Borough,,
Central Toronto,9,9
Downtown Toronto,19,19
East Toronto,5,5
East York,5,5
Etobicoke,12,12
Mississauga,1,1
North York,24,24
Not assigned,77,77
Scarborough,17,17


There are 77 `Not assigned` boroughs. We drop those from the dataframe.

In [80]:
df = df[df['Borough'] != 'Not assigned']

df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [81]:
# Reset the index to start at 0.
df.reset_index(drop=True, inplace=True)
df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [35]:
# The shape of the resulting dataframe
df.shape

(103, 3)

## Section 2: Adding latitude and longitude to the dataframe

In [40]:
# import the geospatial data csv

geodata = pd.read_csv('Geospatial_Coordinates.csv')
geodata.shape

(103, 3)

This matches the size of the dataframe. Good! Let's join the two tables. First we check that the Postal Code IDs match up.

In [68]:
lista=list(df['Postal Code'].sort_values())
listb=list(geodata['Postal Code'].sort_values())
lista==listb

True

This postal codes match! 

In [82]:
# Join the original dataframe and the geodata dataframe.
df = df.join(geodata.set_index('Postal Code'), on='Postal Code')
df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


This appears to be what we want for problem 2 of this assignment.

## Section 3: Cluster the neighborhoods